In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, initializers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import joblib


In [48]:
dataset = 2

hr_file = f"D:/Ankit work/irs data/data{dataset}/hr.csv"
hd_file = f"D:/Ankit work/irs data/data{dataset}/hd.csv"
G_file = f"D:/Ankit work/irs data/data{dataset}/G.csv"
W_file = f"D:/Ankit work/irs data/notebooks/only_w/predictions.csv"
Theta_file = f"D:/Ankit work/irs data/data{dataset}/theta.csv"

scaler_X_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/scaler_X.pkl"
scaler_Y_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/scaler_Y.pkl"

temp_scaler_X_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/temp_scaler_X.pkl"
temp_scaler_Y_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/temp_scaler_Y.pkl"

temp_modal_save_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/my_trained_model_temp.keras"
modal_save_file = "D:/Ankit work/irs data/notebooks/w_followed_by_theta/my_trained_model.keras"

In [49]:
# Load input datasets
hr = pd.read_csv(hr_file, header=None)
hd = pd.read_csv(hd_file, header=None)
G = pd.read_csv(G_file, header=None)

# Load output datasets
W = pd.read_csv(W_file, header=None)
Theta = pd.read_csv(Theta_file, header=None)

# Concatenate input datasets (hr, hd, G) and output datasets (W, theta)
X = pd.concat([hr, hd, G, W], axis=1)
Y = pd.concat([Theta], axis=1)

# Standardize input data
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Optionally, scale output data (depending on your needs)
scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y)



In [50]:
print(X.shape, X_scaled.shape)
print(Y.shape, Y_scaled.shape)

(876650, 158) (876650, 158)
(876650, 50) (876650, 50)


In [51]:
# Manually shuffle the dataset using pandas
X_shuffled = pd.DataFrame(X_scaled).sample(frac=1, random_state=42).reset_index(drop=True)
Y_shuffled = pd.DataFrame(Y_scaled).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into train, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X_scaled, Y_scaled, test_size=0.4, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

print(X_train.shape, Y_train.shape)

(525990, 158) (525990, 50)


In [6]:
l2_lambda = 0.00001
dropout_rate = 0.001  # Set the dropout rate (0.2 = 20%)

In [7]:
# Define the neural network
model = models.Sequential()

# Input layer (shape based on input features)
model.add(layers.InputLayer(shape=(X_train.shape[1],)))

# Hidden layers
model.add(layers.Dense(314, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.Dropout(dropout_rate)) 
model.add(layers.Dense(256, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.Dropout(dropout_rate)) 
model.add(layers.Dense(128, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.Dropout(dropout_rate)) 
model.add(layers.Dense(100, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.Dropout(dropout_rate)) 
model.add(layers.Dense(70, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))
model.add(layers.Dropout(dropout_rate)) 
model.add(layers.Dense(64, activation='relu', kernel_initializer=initializers.HeNormal(),  kernel_regularizer=regularizers.l2(l2_lambda)))

# Output layer
model.add(layers.Dense(Y_train.shape[1]))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [8]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Create the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

In [9]:
train_count = 1

In [10]:
history = model.fit(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val), batch_size=128, shuffle=True, callbacks=[early_stopping])

Epoch 1/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 0.9113 - mae: 0.8270 - val_loss: 0.4566 - val_mae: 0.5217
Epoch 2/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.3862 - mae: 0.4684 - val_loss: 0.2635 - val_mae: 0.3732
Epoch 3/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.2600 - mae: 0.3687 - val_loss: 0.2316 - val_mae: 0.3442
Epoch 4/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.2306 - mae: 0.3422 - val_loss: 0.2177 - val_mae: 0.3311
Epoch 5/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - loss: 0.2168 - mae: 0.3294 - val_loss: 0.2038 - val_mae: 0.3179
Epoch 6/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.2072 - mae: 0.3206 - val_loss: 0.1976 - val_mae: 0.3122
Epoch 7/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.2016 - mae: 0.3156 - val_loss: 0.1966 - val_mae: 0.3114
Epoch 8/100
4110/4110 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.1971 - mae: 0.3116 - val_loss: 0.1898 - val_mae: 0.3051
Epoch 9/100
4110/4110 ━━

In [11]:

# Evaluate the model on training, validation, and test sets
train_loss, train_mae = model.evaluate(X_train, Y_train)
val_loss, val_mae = model.evaluate(X_val, Y_val)
test_loss, test_mae = model.evaluate(X_test, Y_test)

# Get model predictions for calculating MAPE
# train_pred = model.predict(X_train)
# val_pred = model.predict(X_val)
# test_pred = model.predict(X_test)


# Create a remark about model performance (e.g., overfitting or good fit)
if train_loss < val_loss and train_loss < test_loss:
    remark = "Possible overfitting detected."
elif val_loss <= test_loss:
    remark = "Good fit."
else:
    remark = "Model underfits the data."
    
model.save(temp_modal_save_file)  # Save as HDF5
# Save the scaler to a file
joblib.dump(scaler_X, temp_scaler_X_file)
joblib.dump(scaler_Y, temp_scaler_Y_file)


16438/16438 ━━━━━━━━━━━━━━━━━━━━ 9s 520us/step - loss: 0.1573 - mae: 0.2761
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 524us/step - loss: 0.1594 - mae: 0.2779
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 520us/step - loss: 0.1573 - mae: 0.2763


['D:/Ankit work/irs data/notebooks/w_followed_by_theta/temp_scaler_Y.pkl']

In [12]:

# Prepare the text to log the errors and remarks
log_text = (f"Training Loss MSE: {train_loss:.4f}, Validation Loss MSE: {val_loss:.4f}, Test Loss MSE: {test_loss:.4f}\n"
            f"Training MAE: {train_mae:.4f}, Validation MAE: {val_mae:.4f}, Test MAE: {test_mae:.4f}\n"
            f"Remark: {remark}\n"
            f"He intialized , L2 Regularized (0.00001), Dropout (0.001) Early stop (Patience 10),\n" 
            # f"Learning rate scheduling (factor=0.5, patience=10, min_lr=1e-6)\n"
            f"layers: 314, 256, 128, 100, 64\n"
            "-------------------------------------------\n")

# Append the results to a text file
with open('D:/Ankit work/irs data/notebooks/w_followed_by_theta/model_performance_log.txt', 'a') as file:
    file.write(log_text)

print("Training, validation, and test errors saved to 'model_performance_log.txt'.")

Training, validation, and test errors saved to 'model_performance_log.txt'.


### Saving Modal

In [52]:
# Save the trained model to a file (HDF5 format or TensorFlow SavedModel format)
# model.save(modal_save_file)  # Save as HDF5

# Save the scaler to a file
joblib.dump(scaler_X, scaler_X_file)
joblib.dump(scaler_Y, scaler_Y_file)


['D:/Ankit work/irs data/notebooks/w_followed_by_theta/scaler_Y.pkl']

### Loading Modal

In [53]:
from tensorflow.keras.models import load_model
from tensorflow import keras

# Load the model
loaded_model = keras.models.load_model(modal_save_file)

# Load the saved scaler
scaler_X_loaded = joblib.load(scaler_X_file)
scaler_Y_loaded = joblib.load(scaler_Y_file)


In [19]:
loaded_model = model
scaler_X_loaded = scaler_X
scaler_Y_loaded = scaler_Y

In [54]:
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 314)                 │          49,926 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 314)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          80,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          12,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 70)                  │           7,070 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 70)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           4,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 50)                  │           3,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 573,680 (2.19 MB)

 Trainable params: 191,226 (746.98 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 382,454 (1.46 MB)

### PREDICTION

In [58]:
hr_testing_file = f"D:/Ankit work/irs data/testing_data/hr.csv"
hd_testing_file = f"D:/Ankit work/irs data/testing_data/hd.csv"
G_testing_file = f"D:/Ankit work/irs data/testing_data/G.csv"
W_pred_testing_file = f"D:/Ankit work/irs data/notebooks/only_w/testing_data/only_w_pred.csv"

# Load input datasets
hr_testing = pd.read_csv(hr_testing_file, header=None)
hd_testing = pd.read_csv(hd_testing_file, header=None)
G_testing = pd.read_csv(G_testing_file, header=None)
W_pred_testing = pd.read_csv(W_pred_testing_file, header=None)

# Concatenate input datasets (hr, hd, G) and output datasets (W, theta)
X_testing = pd.concat([hr_testing, hd_testing, G_testing, W_pred_testing], axis=1)

X_scaled_testing = scaler_X_loaded.transform(X_testing)
print(X_testing.shape)
print(X_scaled_testing.shape)


(100, 158)
(100, 158)


In [59]:
# Let's assume 'new_data_batch' is a batch of new inputs (e.g., multiple samples).
new_data_batch = X_scaled_testing

# Make predictions for the batch
scaled_predictions = loaded_model.predict(new_data_batch)
predictions = scaler_Y_loaded.inverse_transform(scaled_predictions)

# Print the predictions
print("Predictions for the batch of inputs:", predictions.shape)

# Evaluate the model on test data
test_loss, test_mae = loaded_model.evaluate(X_test, Y_test)

print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predictions for the batch of inputs: (100, 50)
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.1573 - mae: 0.2763
Test Loss: 0.15768949687480927
Test MAE: 0.276504248380661


In [60]:
import pandas as pd

# Assuming Y_pred is a NumPy array
# If it's not, convert it to a NumPy array: Y_pred = np.array(Y_pred)

# Convert to DataFrame
Y_pred_df = pd.DataFrame(predictions)  # Add appropriate column name(s)

# Save to CSV
Y_pred_df.to_csv('D:/Ankit work/irs data/notebooks/w_followed_by_theta/testing_data/theta_after_w_pred.csv', index=False, header=False)  # index=False to avoid adding row numbers to the file


In [23]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file without a header
df = pd.read_csv('D:/Ankit work/irs data/notebooks/w_followed_by_theta/testing_data/predictions.csv', header=None)
df2 = pd.read_csv('D:/Ankit work/irs data/testing_data/W.csv', header=None)

print(df.shape, df2.shape)
# Plot each column separately
for i in range(df.shape[1]):  # df.shape[1] gives the number of columns
    plt.figure()  # Create a new figure for each plot
    plt.plot(df[i])  # Plot the i-th column
    plt.plot(df2[i])  # Plot the i-th column
    plt.title(f'Column {i+1}')  # Add title for clarity
    plt.xlabel('Index')  # X-axis label
    plt.ylabel(f'Column {i+1} Values')  # Y-axis label
    plt.grid(True)  # Enable grid
    plt.show()  # Show the plot


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Ankit work/irs data/notebooks/w_followed_by_theta/testing_data/predictions.csv'